# Network Sniffing
Network sniffing refers to using the network interface on a system to monitor or capture information sent over a wired or wireless connection. An adversary may place a network interface into promiscuous mode to passively access data in transit over the network, or use span ports to capture a larger amount of data.

## Rule Content
```
- title: Network Sniffing
  id: f4d3748a-65d1-4806-bd23-e25728081d01
  status: experimental
  description: Network sniffing refers to using the network interface on a system
    to monitor or capture information sent over a wired or wireless connection. An
    adversary may place a network interface into promiscuous mode to passively access
    data in transit over the network, or use span ports to capture a larger amount
    of data.
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1040/T1040.yaml
  logsource:
    product: linux
    service: auditd
    category: null
  detection:
    selection1:
      type: execve
      a0: tcpdump
      a1: -c
      a3|contains: -i
    selection2:
      type: execve
      a0: tshark
      a1: -c
      a3: -i
    condition: selection1 or selection2
  falsepositives:
  - Legitimate administrator or user uses network sniffing tool for legitimate reason
  level: low
  tags:
  - attack.credential_access
  - attack.discovery
  - attack.t1040

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(type:"execve" AND a1:"\-c" AND ((a0:"tcpdump" AND a3.keyword:*\-i*) OR (a0:"tshark" AND a3:"\-i")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()